In [28]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model, load_model
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model

import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

In [13]:
!nvidia-smi

Fri Jul 19 02:18:31 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 27%   31C    P8    20W / 250W |  10827MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [14]:
print(tf.test.is_gpu_available())

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [15]:
def sanitize(string):
    words = string.split(' ')
    return words

In [16]:
#this is the test set
testTb = Table()
list1 = []
list2 = []

with open("dataset/PAWS_Wiki/train.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        testTb=testTb.with_columns(list1[0][ele], list2[ele])


testX = testTb.select(1, 2)
testY = testTb.select(3)

In [17]:
#this is the training set
tb = Table()
list1 = []
list2 = []

with open("dataset/PAWS_Wiki/train.tsv", 'r', encoding='utf-8') as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        
        list1.append(row) #pairs start at index 1, with id, setence1, sentence2, label, respectively.
     
    for y in range(4):
        temp=[]
        for x in range(1, len(list1)):
            temp.append(list1[x][y])
            
        list2.append(temp)

    for ele in range(len(list2)):
        tb=tb.with_columns(list1[0][ele], list2[ele])
        
len(tb.where("label", are.equal_to('1')).column("label"))


21829

In [18]:
#read in the csv files 


train = pd.read_csv("dataset/PAWS_Wiki/train.tsv", sep='\t')
dev = pd.read_csv("dataset/PAWS_Wiki/dev.tsv", sep='\t')
test = pd.read_csv("dataset/PAWS_Wiki/test.tsv", sep='\t')

print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sentence1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sentence2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sentence1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sentence2'] \
         .apply(sanitize)


test['title1_tokenized'] = \
    test.loc[:, 'sentence1'] \
         .apply(sanitize)
test['title2_tokenized'] = \
    test.loc[:, 'sentence2'] \
         .apply(sanitize)

#train_corpus_x1=tb.select(1)
#train_corpus_x2=tb.select(2)
#test_corpus_x1=testTb.select(1)
#test_corpus_x2=testTb.select(2)
#corpus = pd.concat([train_corpus_x1, train_corpus_x2, test_corpus_x1, test_corpus_x2])

   id                                          sentence1  \
0   1  In Paris , in October 1560 , he secretly met t...   
1   2  The NBA season of 1975 -- 76 was the 30th seas...   
2   3  There are also specific discussions , public p...   
3   4  When comparable rates of flow can be maintaine...   
4   5  It is the seat of Zerendi District in Akmola R...   

                                           sentence2  label  
0  In October 1560 , he secretly met with the Eng...      0  
1  The 1975 -- 76 season of the National Basketba...      1  
2  There are also public discussions , profile sp...      0  
3  The results are high when comparable flow rate...      1  
4  It is the seat of the district of Zerendi in A...      1  


In [19]:
#create a dictionary

MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

train_corpus_x1 = train.title1_tokenized
train_corpus_x2 = train.title2_tokenized
train_corpus = pd.concat([
    train_corpus_x1, train_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)
x1_train = tokenizer \
    .texts_to_sequences(train_corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(train_corpus_x2)


print(tokenizer.document_count)




dev_corpus_x1 = dev.title1_tokenized
dev_corpus_x2 = dev.title2_tokenized
dev_corpus = pd.concat([
    dev_corpus_x1, dev_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x2)


test_corpus_x1 = test.title1_tokenized
test_corpus_x2 = test.title2_tokenized
test_corpus = pd.concat([
    test_corpus_x1, test_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(test_corpus)
tokenizer.texts_to_sequences(test_corpus)
x1_test = tokenizer \
    .texts_to_sequences(test_corpus_x1)
x2_test = tokenizer \
    .texts_to_sequences(test_corpus_x2)

word_index = tokenizer.word_index


print(tokenizer.document_count)



98802
130802


In [20]:
#padding and one-hot

max_seq_len1 = max([
    len(seq) for seq in x1_train])
print(max_seq_len1)   #36

MAX_SEQUENCE_LENGTH = 36  #better to have words covered than uncovered
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)


y_train = train.label

y_train = keras \
    .utils \
    .to_categorical(y_train, num_classes=2)


y_dev = dev.label

y_dev = keras \
    .utils \
    .to_categorical(y_dev, num_classes=2)



y_test = test.label

y_test = keras \
    .utils \
    .to_categorical(y_test, num_classes=2)

x1_test[:3]

36


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   43,
           8,    7,   97,    6, 6066, 5611, 3080,    2,  446,   29, 6910,
           4,    5, 3542,  683,   49, 3108, 1221,    4, 4527, 2922, 3890,
          10,    1,    3],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,   22,  131,
        2253,   10, 1628,    4, 2550,   47,  275,   61,   22,  224,  108,
        3874,    5,  141,    1, 1968,    6,    1,   67,  683,   34, 1742,
          10,  280,    3],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    4,  120,  151,    2,    1, 1180, 1172,  198,
           6, 5608,  905,    2,    2,    1, 3980,  304,   15,   32,  269,
        2334, 2052,    3]], dtype=int32)

In [21]:
print("Training Set")
print("-" * 10)
print(np.shape(x1_train))
print(np.shape(x2_train))
print(np.shape(y_train))
print(np.shape(train_corpus))

print("-" * 10)
print(np.shape(x1_test))
print(np.shape(x2_test))
print(np.shape(y_test))
print(np.shape(test_corpus))

print("Test Set")


Training Set
----------
(49401, 36)
(49401, 36)
(49401, 2)
(98802,)
----------
(8000, 36)
(8000, 36)
(8000, 2)
(16000,)
Test Set


In [11]:
def process_glove_line(line, dim):
    word = None
    embedding = None

    try:
        splitLine = line.split()
        word = " ".join(splitLine[:len(splitLine)-dim])
        embedding = np.array([float(val) for val in splitLine[-dim:]])
    except:
        print(line)

    return word, embedding

def load_glove_model(glove_filepath, dim):
    with open(glove_filepath, encoding="utf8" ) as f:
        content = f.readlines()
        model = {}
        for line in content:
            word, embedding = process_glove_line(line, dim)
            if embedding is not None:
                model[word] = embedding
        return model

glove = load_glove_model("glove.840B.300d.txt", 300)



In [12]:
NUM_EMBEDDING_DIM = 300

embedding_matrix = np.zeros((len(word_index)+1, NUM_EMBEDDING_DIM))


for word, i in word_index.items():
    embedding_vector = glove.get(word)
    
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
#Siamese with GLoVE
NUM_CLASSES=2 #boolean; 1 or 0

 #rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(len(word_index)+1,
                            NUM_EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 5

history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=([x1_test, x2_test], y_test),
    
    shuffle=True
)

NameError: name 'embedding_matrix' is not defined

In [36]:
#Naive Siamese
NUM_CLASSES=2 #boolean; 1 or 0

 #rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

 #this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension




top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM, mask_zero=True)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = LSTM(NUM_LSTM_UNITS)
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('Sentence_Models_H5/siamese.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([x1_test, x2_test], y_test),
    
    shuffle=True
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 36)           0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 36)           0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 36, 300)      3000000     input_15[0][0]                   
                                                                 input_16[0][0]                   
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 128)          219648      embedding_7[0][0]                
          

ResourceExhaustedError: OOM when allocating tensor with shape[36,512,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training_5/Adam/gradients/lstm_7/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3_grad/TensorArrayGatherV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [38]:
!nvidia-smi

Thu Jul 25 02:03:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 28%   34C    P8    20W / 250W |  10827MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [9]:
#BiLSTM?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 300

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM, mask_zero=True)


top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 20

earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_Wiki_GLOVE.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')





Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 36, 300)      3000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [28]:
dot(
    [top_output, bm_output], 
    axes=1, normalize = True)


<tf.Tensor 'dot_2/ExpandDims:0' shape=(?, 1) dtype=float32>

In [26]:
x1_train.shape

(49401, 36)

In [11]:
#cosine similarity


NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 36 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = dot(
    [top_output, bm_output], 
    axes=1, normalize = True)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 20

earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_Wiki_cos_sim.h5', verbose=1, save_best_only=True, monitor='val_loss', mode='min')


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 36)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 36, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [27]:
!nvidia-smi


Fri Jul 19 08:50:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 27%   31C    P8    20W / 250W |  10982MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
        
history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose=1, callbacks=[earlyStopping, mcp_save], validation_data=([x1_dev, x2_dev], y_dev),
    
    shuffle=True
)

Instructions for updating:
Use tf.cast instead.
Train on 49401 samples, validate on 8000 samples
Epoch 1/20
49401/49401 [==============================] - 24s 490us/step - loss: 0.6860 - acc: 0.5548 - val_loss: 0.6893 - val_acc: 0.5221

Epoch 00001: val_loss improved from inf to 0.68929, saving model to BiLSTM_PAWS_Wiki_cos_sim.h5
Epoch 2/20
49401/49401 [==============================] - 22s 444us/step - loss: 0.6400 - acc: 0.7442 - val_loss: 0.6912 - val_acc: 0.5279

Epoch 00002: val_loss did not improve from 0.68929
Epoch 3/20
49401/49401 [==============================] - 22s 442us/step - loss: 0.6024 - acc: 0.8041 - val_loss: 0.6939 - val_acc: 0.5319

Epoch 00003: val_loss did not improve from 0.68929
Epoch 4/20
49401/49401 [==============================] - 22s 443us/step - loss: 0.5679 - acc: 0.8312 - val_loss: 0.6993 - val_acc: 0.5379

Epoch 00004: val_loss did not improve from 0.68929
Epoch 5/20
49401/49401 [==============================] - 22s 443us/step - loss: 0.5309 - acc:

In [34]:
model = load_model("../Sentence_Models_H5/siamese.h5")


OSError: Unable to open file (unable to open file: name = '../Sentence_Models_H5/siamese.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [14]:
predicts = model.predict(
    [x1_train, x2_train])




In [16]:
predict_test = model.predict([x1_test, x2_test])

In [18]:
sen1 = test.sentence1
sen2=test.sentence2

error = []
errr1 = []
errr2 = []
real_cat = []


for x in range(len(predict_test)):
    if (np.argmax(predict_test[x])!=(test.label[x])):
        error = np.append(error, x)
        errr1=np.append(errr1, sen1[x])
        errr2=np.append(errr2, sen2[x])
        real_cat = np.append(real_cat, test.label[x])
print(1-(len(real_cat)/len(x1_test)))

0.507625


In [30]:
test_error_analysis = Table()


test_error_analysis=test_error_analysis.with_column("id", error, "sent1", errr1, "sent2", errr2, "classification", real_cat)
test_error_analysis.show(50)

NameError: name 'error' is not defined

In [45]:


print(len(predicts))

49401


In [17]:
real1=train.sentence1
real2=train.sentence2
realCat=train.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(predicts)):
    if ((np.argmax(predicts[x])!=(train.label[x]))):
        no=np.append(no, x)
        err1=np.append(err1, real1[x])
        err2=np.append(err2, real2[x])
        wrongCat = np.append(wrongCat, realCat[x])
print(1-(len(wrongCat)/len(x1_train)))

0.5772150361328718


In [18]:
train_error_analysis = Table()


train_error_analysis=train_error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "classification", wrongCat)
train_error_analysis.show(30)

id,sent1,sent2,classification
1,The NBA season of 1975 -- 76 was the 30th season of the ...,The 1975 -- 76 season of the National Basketball Associa ...,1
2,"There are also specific discussions , public profile deb ...","There are also public discussions , profile specific dis ...",0
3,"When comparable rates of flow can be maintained , the re ...",The results are high when comparable flow rates can be m ...,1
4,It is the seat of Zerendi District in Akmola Region .,It is the seat of the district of Zerendi in Akmola region .,1
5,William Henry Henry Harman was born on 17 February 1828 ...,"William Henry Harman was born in Waynesboro , Virginia o ...",1
7,With a discrete amount of probabilities Formula 1 with t ...,Given a discrete set of probabilities formula _ 1 with t ...,1
12,The city sits at the confluence of the Snake River with ...,The city lies at the confluence of the Snake River and t ...,1
14,The Austrian school assumes that the subjective choices ...,The Austrian school assumes that the subjective choices ...,0
15,Werder 's forces invested Belfort and reached the city o ...,Werder 's troops invested Belfort and reached the city o ...,1
17,The first five weapons were delivered in the first half ...,The first five weapons were delivered in the first half ...,1


In [19]:
false_negative = train_error_analysis.where("classification", are.equal_to(1))
false_negative.show(20)

#倒裝句
#formula
#more of prepositional phrases/verbs instead of the changing of order that is causing the misclassification

id,sent1,sent2,classification
1,The NBA season of 1975 -- 76 was the 30th season of the ...,The 1975 -- 76 season of the National Basketball Associa ...,1
3,"When comparable rates of flow can be maintained , the re ...",The results are high when comparable flow rates can be m ...,1
4,It is the seat of Zerendi District in Akmola Region .,It is the seat of the district of Zerendi in Akmola region .,1
5,William Henry Henry Harman was born on 17 February 1828 ...,"William Henry Harman was born in Waynesboro , Virginia o ...",1
7,With a discrete amount of probabilities Formula 1 with t ...,Given a discrete set of probabilities formula _ 1 with t ...,1
12,The city sits at the confluence of the Snake River with ...,The city lies at the confluence of the Snake River and t ...,1
15,Werder 's forces invested Belfort and reached the city o ...,Werder 's troops invested Belfort and reached the city o ...,1
17,The first five weapons were delivered in the first half ...,The first five weapons were delivered in the first half ...,1
19,The friendship between him and Duncan ended at a club me ...,The friendship between him and Duncan ended in 1951 at a ...,1
22,Shaffer Creek is a tributary of the Raystown Branch Juni ...,Shaffer Creek is an tributary of Brush Creek ( Raystown ...,1


In [20]:
false_positive = train_error_analysis.where("classification", are.equal_to(0))
false_positive
#is/has, address scrambling


id,sent1,sent2,classification
2,"There are also specific discussions , public profile deb ...","There are also public discussions , profile specific dis ...",0
14,The Austrian school assumes that the subjective choices ...,The Austrian school assumes that the subjective choices ...,0
35,"In addition to Michael Boddicker , and Patrick Moraz , t ...","In addition to Diana Hubbard , the album contains musica ...",0
135,"His parents are Ernie Gordon , who grew up with Yankees ...","His parents are Wendy Abrahamsen , who grew up a Yankees ...",0
142,Fish species of the parks rivers and lakes are Sperchios ...,Fish species of the parks rivers and lakes are Sperchios ...,0
155,Neyab ( also Romanized as Neyab ) is a village in Esfara ...,Neyab ( also romanized as Neyab ) is a village in the di ...,0
196,Delaware County is a city in and the county town of Dela ...,Delaware is a city in and the county seat of Delaware Co ...,0
247,Cornelis van Cleve painted primarily mythological painti ...,Cornelis van Cleve painted mainly religious paintings an ...,0
248,Their skin also secretes chemicals that are poisonous an ...,"Also , their skin secretes chemicals that are distastefu ...",0
251,"In rats , it also inhibits the central , if not peripher ...","It also inhibits the peripheral , if not central , secre ...",0


In [21]:
#"when...", split into two sentences, when two sentences are equal,  
#mostly false positives

In [22]:
#embedding layer output dimension is arbitrary?




In [23]:
sanitize("I like, hue")

['I', 'like,', 'hue']

In [24]:
!nvidia-smi

Tue Jun 25 03:32:57 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   47C    P8     8W / 200W |   7772MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [25]:
#possible optimization strats:

#tokenization without punctuation aka modify the dictionary
#tweak MAX_SEQUENCE_LENGTH 
#masking
#NUM_EMBEDDING_DIMENSION
#"optimizer" in the compile function
#loss function
#a different activation function?
#sth other than word level tokenization?
#train on other databases?
#pretrained word embedding
#check errors on the test set?
#model attention seq2seq
#GLOVE dimension modification
#more sanitization?

In [26]:
# model adaptation: https://leemeng.tw/shortest-path-to-the-nlp-world-a-gentle-guide-of-natural-language-processing-and-deep-learning-for-everyone.html#%E6%B7%B1%E5%BA%A6%E5%AD%B8%E7%BF%92-3-%E6%AD%A5%E9%A9%9F

In [27]:
#https://paperswithcode.com/paper/multi-task-deep-neural-networks-for-natural


In [28]:
#attempts: with self trained word embedder, glove embedder, zero masking, 

In [29]:
# We randomly select 5,000 paraphrases and 5,000 non-paraphrases as the dev set, and sample
#another 5,000 paraphrases and 5,000 non-paraphrases as the
#test set. We keep the remaining instances as the training set 2